In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession


In [ ]:
sc = SparkContext.getOrCreate()

In [ ]:
spark = SparkSession.builder.appName('SQL').config('spark.som.config.option', 'some-value').getOrCreate()

In [ ]:
from pyspark.streaming import StreamingContext

In [ ]:
streaming = StreamingContext(sc, 5)

In [ ]:
lines = streaming.socketTextStream('localhost', 9999)

In [ ]:
words = lines.flatMap(lambda line : line.split())
pairs = words.map(lambda w :(w,1))
count = pairs.reduceByKey(lambda a,b : a+b)
count.pprint()

In [ ]:
streaming.start()

In [ ]:
streaming.awaitTermination()

In [ ]:
lines = spark.readStream.format('socket').option('host', 'localhost').option('port', 9999).load()

In [ ]:
lines

In [ ]:
from pyspark.sql.functions import explode, split

In [ ]:
type(lines)

In [ ]:
words = lines.select(
    explode(split(lines.value, ' ')).alias('word')

)

In [ ]:
wordCount = words.groupBy('word').count()

In [ ]:
stream = wordCount.writeStream.outputMode('complete').format('console').start()


In [ ]:
stream.awaitTermination()

In [3]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [34]:
spark  = SparkSession.builder.getOrCreate()

In [37]:
df = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 144.6, 5.4, 35, 'M'),
    (3, 144.7, 5.6, 37, 'F'),
    (4, 144.8, 5.3, 31, 'M'),
    (5, 144.9, 5.7, 29, 'M'),
    (6, 145.0, 5.8, 30, 'F'),
    (7, 145.1, 5.1, 39, 'F'),
    (1, 144.5, 5.9, 33, 'M')
    
    ],
    ['id', 'weight', 'height', 'age', 'gender']
)

In [9]:
df.count(), df.distinct().count()

(8, 7)

In [10]:
df = df.dropDuplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  4| 144.8|   5.3| 31|     M|
|  3| 144.7|   5.6| 37|     F|
|  1| 144.5|   5.9| 33|     M|
|  6| 145.0|   5.8| 30|     F|
|  5| 144.9|   5.7| 29|     M|
|  2| 144.6|   5.4| 35|     M|
|  7| 145.1|   5.1| 39|     F|
+---+------+------+---+------+



In [11]:
df.orderBy('id').show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 144.6|   5.4| 35|     M|
|  3| 144.7|   5.6| 37|     F|
|  4| 144.8|   5.3| 31|     M|
|  5| 144.9|   5.7| 29|     M|
|  6| 145.0|   5.8| 30|     F|
|  7| 145.1|   5.1| 39|     F|
+---+------+------+---+------+



In [12]:
df.count(), df.distinct().count()

(7, 7)

In [13]:
df.orderBy('id').show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 144.6|   5.4| 35|     M|
|  3| 144.7|   5.6| 37|     F|
|  4| 144.8|   5.3| 31|     M|
|  5| 144.9|   5.7| 29|     M|
|  6| 145.0|   5.8| 30|     F|
|  7| 145.1|   5.1| 39|     F|
+---+------+------+---+------+



In [19]:
df = df.dropDuplicates([c for c in df.columns if c == 'id'])

In [20]:
df

DataFrame[id: bigint, weight: double, height: double, age: bigint, gender: string]

In [21]:
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  7| 145.1|   5.1| 39|     F|
|  6| 145.0|   5.8| 30|     F|
|  5| 144.9|   5.7| 29|     M|
|  1| 144.5|   5.9| 33|     M|
|  3| 144.7|   5.6| 37|     F|
|  2| 144.6|   5.4| 35|     M|
|  4| 144.8|   5.3| 31|     M|
+---+------+------+---+------+



In [22]:
from pyspark.sql import functions as f

In [24]:
df.agg(
    f.count('id'),
    f.countDistinct('id')
    
).show()

+---------+------------------+
|count(id)|count(DISTINCT id)|
+---------+------------------+
|        7|                 7|
+---------+------------------+



In [26]:
df.agg(
    f.count('id').alias('Count'),
    f.countDistinct('id').alias('DCouont')
    
).show()

+-----+-------+
|Count|DCouont|
+-----+-------+
|    7|      7|
+-----+-------+



In [27]:
df.withColumn('nid', f.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|          nid|
+---+------+------+---+------+-------------+
|  7| 145.1|   5.1| 39|     F|  94489280512|
|  6| 145.0|   5.8| 30|     F| 386547056640|
|  5| 144.9|   5.7| 29|     M| 584115552256|
|  1| 144.5|   5.9| 33|     M| 592705486848|
|  3| 144.7|   5.6| 37|     F| 919123001344|
|  2| 144.6|   5.4| 35|     M|1099511627776|
|  4| 144.8|   5.3| 31|     M|1202590842880|
+---+------+------+---+------+-------------+



In [28]:
df.withColumn('oid', df.id).show()

+---+------+------+---+------+---+
| id|weight|height|age|gender|oid|
+---+------+------+---+------+---+
|  7| 145.1|   5.1| 39|     F|  7|
|  6| 145.0|   5.8| 30|     F|  6|
|  5| 144.9|   5.7| 29|     M|  5|
|  1| 144.5|   5.9| 33|     M|  1|
|  3| 144.7|   5.6| 37|     F|  3|
|  2| 144.6|   5.4| 35|     M|  2|
|  4| 144.8|   5.3| 31|     M|  4|
+---+------+------+---+------+---+



In [31]:
df = df.withColumn('oid', f.monotonically_increasing_id())

In [32]:
df.orderBy('oid').show()

+-------------+------+------+---+------+-------------+
|           id|weight|height|age|gender|          oid|
+-------------+------+------+---+------+-------------+
|  94489280512| 145.1|   5.1| 39|     F|  94489280512|
| 386547056640| 145.0|   5.8| 30|     F| 386547056640|
| 584115552256| 144.9|   5.7| 29|     M| 584115552256|
| 592705486848| 144.5|   5.9| 33|     M| 592705486848|
| 919123001344| 144.7|   5.6| 37|     F| 919123001344|
|1099511627776| 144.6|   5.4| 35|     M|1099511627776|
|1202590842880| 144.8|   5.3| 31|     M|1202590842880|
+-------------+------+------+---+------+-------------+



In [36]:
dF = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 144.6, 5.4, 35, None),
    (3, 144.7, 5.6, 37, None),
    (4, 144.8, 5.3, 31, 'M'),
    (5, 144.9, 5.7, 29, 'M'),
    (6, 145.0, 5.8, 30, 'F'),
    (7, 145.1, 5.1, 39, 'F'),
    (1, 144.5, 5.9, 33, 'M'),
    (2, 312, 5, 59, 'F'),
    (2, 31, 6, 33, 'M'),
    (8, None, None, None, None)
    
    ],
    ['id', 'weight', 'height', 'age', 'gender']
)

TypeError: field weight: Can not merge type <class 'pyspark.sql.types.DoubleType'> and <class 'pyspark.sql.types.LongType'>

In [38]:
df = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 144.6, 5.4, 35, 'M'),
    (3, 144.7, 5.6, 37, 'F'),
    (4, 144.8, 5.3, 31, 'M'),
    (5, 144.9, 5.7, 29, 'M'),
    (6, 145.0, 5.8, 30, 'F'),
    (7, 145.1, 5.1, 39, 'F'),
    (1, 144.5, 5.9, 33, 'M')
    
    ],
    ['id', 'weight', 'height', 'age', 'gender']
)